# 公式Tutorialの実行

## 手順の概要

Watson NLC 公式ドキュメントからの引用です。

https://www.ibm.com/watson/developercloud/doc/natural-language-classifier/getting-started.html

<img src="images/oper0217_03.png" />

### (1) 認証情報取得

- Bluemixにログインし、NLCサービスを作成
- サービス作成時に発行される認証情報（Credentials）を使用します

上記の手順は<b><a href="Watson_NLC_00.ipynb">こちら</a></b>になります。

### (2) 分類器作成

- サンプルの訓練データ（CSV）をダウンロード
- 日本語に変えてみます <---公式ページにない手順
- 分類器作成を開始させるAPI（POST /classifiers/）を実行

分類器の作成開始から完了までは時間を要します。

分類器作成が完了したかどうかは、完了したかどうかを照会するAPIで、逐一問い合わせる必要があります。

### (3) 分類実行

- 訓練が完了した分類器に対し、分類したいテキストを送信
- 該当する上位クラスと、マッチ率（confidence）がJSON形式で戻ります

該当する上位１０件までがリストされるようです。

またTutorialの例では、訓練データに入っていない単語が含まれているテキストでも、分類が可能な旨の記述がありますが・・・

### (4) 分類器の削除

- 上記までの試行が完了したら、分類器を削除

無料で試行できる分類器インスタンスが１件なので、忘れないうちに削除しておきましょう・・・ということなのかと思われます。

## 依存ライブラリー導入

Python3からNLCのAPIをコールするためには、依存ライブラリー「watson_developer_cloud」を追加導入する必要がございます。

コマンドラインから「pip3 install watson_developer_cloud」と実行し、導入できるかと存じます。

~~~~
MacBookPro-makmorit-jp:~ makmorit$ pip3 install watson_developer_cloud
Collecting watson-developer-cloud
  Downloading watson-developer-cloud-0.23.0.tar.gz (52kB)
    100% |████████████████████████████████| 61kB 451kB/s 
Requirement already satisfied (use --upgrade to upgrade): requests<3.0,>=2.0 in /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages (from watson-developer-cloud)
Collecting pysolr<4.0,>=3.3 (from watson-developer-cloud)
  Downloading pysolr-3.6.0-py2.py3-none-any.whl
Building wheels for collected packages: watson-developer-cloud
  Running setup.py bdist_wheel for watson-developer-cloud ... done
  Stored in directory: /Users/makmorit/Library/Caches/pip/wheels/e2/e9/4e/bdd7ab2ed130d3ed16ac583d4843de1335f7f03f531d6ba01d
Successfully built watson-developer-cloud
Installing collected packages: pysolr, watson-developer-cloud
Successfully installed pysolr-3.6.0 watson-developer-cloud-0.23.0
You are using pip version 8.1.1, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
MacBookPro-makmorit-jp:~ makmorit$
~~~~

## 実行例

上記のライブラリー「watson_developer_cloud」を使用し、Python3でTutorialを実行してみます。

もともとWeb APIの戻り値がJSON形式であるため、Python3との親和性は高いかと存じます。

### クラス「NaturalLanguageClassifierV1」生成

生成時に、サービス作成時に発行された認証情報を設定します。

In [1]:
import json
from watson_developer_cloud import NaturalLanguageClassifierV1

natural_language_classifier = NaturalLanguageClassifierV1(
  username='7fdb13f0-1c16-48f5-9a7d-6374ee56925d',
  password='LLwyqghs3cPB')

すでに作成されている分類器を確認するには、list関数を使用します。

（いまはまだ作成されていないので、空リストが戻ります）

In [2]:
classifiers = natural_language_classifier.list()
classifiers

{'classifiers': []}

### 訓練データの作成

例を簡単にするのと「日本語もサポート」と謳っている仕様の検証のため、ダウンロードしたサンプルを日本語に書き換えます。

#### ダウンロードしたサンプルデータ

~~~~
How hot is it today?,temperature
Is it hot outside?,temperature
Will it be uncomfortably hot?,temperature
：
What is today's expected humidity?,conditions
Will the blizzard hit us?,conditions
Is it drizzling?,conditions
~~~~



#### 日本語化したサンプルデータ

形態素解析が必要となります（御社テストデータを、MeCab にかけた例。仕込み用コードは<b><a href="Watson_NLC_99.ipynb">こちら</a></b>）

~~~~
：
"試用 期間 は ある ？ ",2657
"試用 期間 は ？ ",2657
"試用 期間 は 有り ます か ？ ",2657
"初期 費用 月額 費用 以外 に 掛かる 費用 は ？ ",2729
"初期 費用 、 月額 費用 以外 は ？ ",2729
"初期 費用 、 月額 費用 以外 に かかり ます か ？ ",2729
"導入 を 考え てる ので 、 デモ を 見せ て 下さい ",2658
"導入 し たい の です が 、 デモ は あり ます か ？ ",2658
"導入 を 考え て いる の です が 、 デモ を 見せ て いただく こと は でき ます か ？ ",2658
"お客様 用 です か ？ ",2727
~~~~

### 分類器作成

create関数によりAPIコールを実行すると、分類器インスタンスが、Bluemix上に１件生成されます。

（API呼出件数／インスタンス件数は、いずれも課金対象となるので、無償で評価時は実行回数に注意）

In [3]:
test_data_file = 'resources/train_myope_info_conversation.csv'
f = open(test_data_file, 'rt') # これは確認用
print(f.read()[:100])
f.close()

"My - ope ってな ん です か ？ ",2653
"どんな こと が 出来る の ？ ",2654
"どう やっ て 会話 を 覚え させる の ？ ",2655
"どんな 企業 が 使っ 


In [4]:
''' 後日、実行予定です
classifier_id = None
status = None
with open(test_data_file, 'rb') as training_data:
  classifier = natural_language_classifier.create(
    training_data=training_data,
    name='My Classfier',
    language='ja'
  )
  classifier_id = classifier['classifier_id']
  status = classifier['status']
'''

" 後日、実行予定です\nclassifier_id = None\nstatus = None\nwith open(test_data_file, 'rb') as training_data:\n  classifier = natural_language_classifier.create(\n    training_data=training_data,\n    name='My Classfier',\n    language='ja'\n  )\n  classifier_id = classifier['classifier_id']\n  status = classifier['status']\n"

### 分類器作成状況の照会

分類器インスタンスの生成には、時間がかかるため、作成が完了したかどうかを知るには、下記のような状況照会のためのAPIコールを実行する必要がございます。

（API呼出件数は課金対象となるので、無償で評価時は実行回数に注意）

#### 分類器作成が利用できるまで待機

以下の例では、最大３０分間待機します。

In [5]:
''' 後日、実行予定です
from time import sleep
import sys
i = 0
while i < 30:
    statusDict = natural_language_classifier.status(classifier_id)
    status = statusDict['status']
    if status == 'Available':
        break

    print('Classifier [%s] now training, please wait...' % classifier_id)
    sys.stdout.flush()
    sleep(60)
    i += 1

if status == 'Available':
    print('Classifier [%s] training completed.' % classifier_id)
else:
    print('Classifier [%s] training failed.' % classifier_id)
'''

" 後日、実行予定です\nfrom time import sleep\nimport sys\ni = 0\nwhile i < 30:\n    statusDict = natural_language_classifier.status(classifier_id)\n    status = statusDict['status']\n    if status == 'Available':\n        break\n\n    print('Classifier [%s] now training, please wait...' % classifier_id)\n    sys.stdout.flush()\n    sleep(60)\n    i += 1\n\nif status == 'Available':\n    print('Classifier [%s] training completed.' % classifier_id)\nelse:\n    print('Classifier [%s] training failed.' % classifier_id)\n"

### 分類実行

classify関数によりAPIコールを実行します。

引数のテキストが分類され、クラスに対するマッチ率が、上位１０クラス分戻ります。

（API呼出件数は課金対象となるので、無償で評価時は実行回数に注意）

In [6]:
''' 後日、実行予定です
question_text = '今日の気温は何度になるでしょうか？'
classesDict = natural_language_classifier.classify(classifier_id, question_text)
'''

" 後日、実行予定です\nquestion_text = '今日の気温は何度になるでしょうか？'\nclassesDict = natural_language_classifier.classify(classifier_id, question_text)\n"

#### 分類結果の参照

分類されたクラスとマッチ率を表示します。

In [7]:
''' 後日、実行予定です
for c in classesDict['classes']:
    print('class name=%s, confidence=%0.1f%%' % (c['class_name'], float(c['confidence'])*100.0))
'''

" 後日、実行予定です\nfor c in classesDict['classes']:\n    print('class name=%s, confidence=%0.1f%%' % (c['class_name'], float(c['confidence'])*100.0))\n"

### 分類器削除

remove関数によりAPIコールを実行すると、分類器インスタンスがBluemix上から削除されます。

（API呼出件数は課金対象となるので、無償で評価時は実行回数に注意）

In [8]:
''' 後日、実行予定です
natural_language_classifier.remove(classifier_id)
print('Classifier [%s] removed.' % classifier_id)
'''

" 後日、実行予定です\nnatural_language_classifier.remove(classifier_id)\nprint('Classifier [%s] removed.' % classifier_id)\n"